In [26]:
# %%writefile requirements.txt
# joblib~=1.0
# numpy==1.23.1
# mxnet-mkl==1.6.0
# scikit-learn
# google-cloud-storage<3.0.0dev,>=1.32.0

Overwriting requirements.txt


In [27]:
# ! pip install -U  -q -r requirements.txt

In [58]:
! pip install -U  -q uvicorn[standard] fastapi

In [59]:
! pip install --upgrade --quiet  google-cloud-aiplatform \
                                 google-cloud-storage

In [14]:
PROJECT_ID = "ai-hangsik" 
LOCATION = "us-central1" 
BUCKET_URI = f"gs://sllm_0116" 

In [61]:
import os
import sys
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [1]:
!cd /home/jupyter/custom_container

In [2]:
!pwd

/home/jupyter/custom_container


In [3]:
MODEL_ARTIFACT_DIR = "custom-container-prediction-model"
REPOSITORY = "custom-container-prediction"
IMAGE = "sklearn-fastapi-server"
MODEL_DISPLAY_NAME = "sklearn-custom-container"

In [4]:
%mkdir app

In [6]:
%mv model.pickle app

In [12]:
%cd /home/jupyter/custom_container

/home/jupyter/custom_container


In [15]:
!gsutil cp app/model.pickle {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://app/model.pickle [Content-Type=application/octet-stream]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      


In [16]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import joblib
import json
import numpy as np
import pickle
import os

from google.cloud import storage
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris

app = FastAPI()
gcs_client = storage.Client()

# with open("model.pickle", 'wb') as model_f:
#     gcs_client.download_blob_to_file(
#         f"{os.environ['AIP_STORAGE_URI']}/preprocessor.pkl", preprocessor_f
#     )
#     gcs_client.download_blob_to_file(
#         f"{os.environ['AIP_STORAGE_URI']}/model.joblib", model_f
#     )

with open("model.pickle", "rb") as f:
    model = pickle.load(f)

_class_names = load_iris().target_names
_model = model
# _preprocessor = preprocessor

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    # preprocessed_inputs = _preprocessor.preprocess(inputs)
    outputs = _model.predict(inputs)

    return {"predictions": [_class_names[class_num] for class_num in outputs]}


Writing app/main.py


In [17]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing app/prestart.sh


In [18]:
%%writefile instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

Writing instances.json


In [ ]:
# %%writefile requirements.txt
# joblib~=1.0
# numpy==1.23.1
# mxnet-mkl==1.6.0
# scikit-learn
# google-cloud-storage<3.0.0dev,>=1.32.0

In [19]:
%%writefile requirements.txt
numpy
scikit-learn
google-cloud-storage

Writing requirements.txt


In [20]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

Writing Dockerfile


In [21]:
! docker build --tag="{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}" .

Sending build context to Docker daemon  305.2kB
Step 1/4 : FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9
 ---> 95735a0480b5
Step 2/4 : COPY ./app /app
 ---> bfbeee204d5d
Step 3/4 : COPY requirements.txt requirements.txt
 ---> ad52193287db
Step 4/4 : RUN pip install -r requirements.txt
 ---> Running in cc0da01fb54e
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
!docker stop local-iris

local-iris


In [23]:
!docker rm local-iris

local-iris


In [24]:
! docker ps -a

CONTAINER ID   IMAGE                                                                           COMMAND                  CREATED        STATUS                    PORTS     NAMES
d7598bff538e   us-central1-docker.pkg.dev/ai-hangsik/custom-inference-gpu/tgi-release:latest   "./entrypoint.sh '--…"   47 hours ago   Exited (0) 47 hours ago             stupefied_williams


In [25]:
! docker images

REPOSITORY                                                                                                                TAG         IMAGE ID       CREATED             SIZE
us-central1-docker.pkg.dev/ai-hangsik/custom-container-prediction/sklearn-fastapi-server                                  latest      afbfba19495d   35 seconds ago      1.4GB
<none>                                                                                                                    <none>      e5aa46fa9e71   54 minutes ago      1.75GB
<none>                                                                                                                    <none>      0b57ae845577   About an hour ago   1.75GB
<none>                                                                                                                    <none>      087a9d9cfa38   3 hours ago         1.75GB
tiangolo/uvicorn-gunicorn-fastapi                                                                                         p

In [26]:
! docker run -d -p 80:8080 \
        --name=local-iris \
        -e AIP_HTTP_PORT=8080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
        "{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

dd37ac4fb2799c5321b63d86136d2df0794235a84985940bdf4ec9fb01cb1b3b


In [32]:
! docker ps -a

CONTAINER ID   IMAGE                                                                                      COMMAND                  CREATED         STATUS                    PORTS                                   NAMES
dd37ac4fb279   us-central1-docker.pkg.dev/ai-hangsik/custom-container-prediction/sklearn-fastapi-server   "/start.sh"              4 minutes ago   Up 4 minutes              0.0.0.0:80->8080/tcp, :::80->8080/tcp   local-iris
d7598bff538e   us-central1-docker.pkg.dev/ai-hangsik/custom-inference-gpu/tgi-release:latest              "./entrypoint.sh '--…"   47 hours ago    Exited (0) 47 hours ago                                           stupefied_williams


In [29]:
!docker logs dd37ac4fb279

Checking for script in /app/prestart.sh
Running script /app/prestart.sh
{"loglevel": "info", "workers": 24, "bind": "0.0.0.0:8080", "graceful_timeout": 120, "timeout": 120, "keepalive": 5, "errorlog": "-", "accesslog": "-", "workers_per_core": 1.0, "use_max_workers": null, "host": "0.0.0.0", "port": "8080"}
[2025-02-03 07:45:29 +0000] [1] [INFO] Starting gunicorn 23.0.0
[2025-02-03 07:45:29 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2025-02-03 07:45:29 +0000] [1] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2025-02-03 07:45:29 +0000] [7] [INFO] Booting worker with pid: 7
[2025-02-03 07:45:29 +0000] [8] [INFO] Booting worker with pid: 8
[2025-02-03 07:45:29 +0000] [9] [INFO] Booting worker with pid: 9
[2025-02-03 07:45:29 +0000] [10] [INFO] Booting worker with pid: 10
[2025-02-03 07:45:29 +0000] [11] [INFO] Booting worker with pid: 11
[2025-02-03 07:45:29 +0000] [12] [INFO] Booting worker with pid: 12
[2025-02-03 07:45:30 +0000] [13] [INFO] Booting worker with pid: 

In [30]:
! curl localhost/health

{}

In [31]:
! curl -X POST \
  -d @instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

{"predictions":["versicolor","setosa"]}